In [50]:
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import os

In [51]:
os.chdir(r"C:\Users\gsyoume\Desktop\KIS DATA WAREHOUESE\pre-processing\test_excel\other\NLP-with-Python")

In [52]:
df = pd.read_csv('Seattle_Hotels_Duplicates - Copy2.csv', encoding="latin-1")

In [53]:
df.head(5)

,id,name,address
0,a,Hilton Garden Inn Seattle Downtown,"1821 Boren Avenue, Seattle Washington 98101 USA"
1,0,Sheraton Grand Seattle,"1400 6th Avenue, Seattle, Washington 98101 USA"
2,0,Crowne Plaza Seattle Downtown,"1113 6th Ave, Seattle, WA 98101"
3,0,Kimpton Hotel Monaco Seattle,"1101 4th Ave, Seattle, WA98101"
4,0,The Westin Seattle,"1900 5th Avenue, Seattle, Washington 98101 USA"


In [54]:
df.name.value_counts()

Seattle Inn Northgate                              2
Ace Hotel Seattle                                  2
Hotel Seattle                                      2
Comfort Inn & Suites                               2
citizenM Seattle South Lake Union hotel            2
                                                  ..
Hotel Hotel                                        1
Days Inn Seattle South Tukwila                     1
Mediterranean Inn                                  1
Best Western Plus Pioneer Square Hotel Downtown    1
The Paramount Hotel Seattle                        1
Name: name, Length: 155, dtype: int64

In [55]:
df.loc[df['name'] == 'Roy Street Commons']

,id,name,address
82,0,Roy Street Commons,"621 12th Ave E, Seattle, WA 98102"
90,0,Roy Street Commons,"621 12th Avenue East, Seattle, Washington 98102"


In [56]:
df['name_address'] = df['name'] + ' ' + df['address']+ '_'+ df['id']
name_address = df['name_address']


In [57]:
vectorizer = TfidfVectorizer("char", ngram_range=(1, 4), sublinear_tf=True)
tf_idf_matrix = vectorizer.fit_transform(name_address)

In [58]:
tf_idf_matrix

<168x3236 sparse matrix of type '<class 'numpy.float64'>'
	with 6006 stored elements in Compressed Sparse Row format>

In [59]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
  
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 5)

In [60]:
matches

<168x168 sparse matrix of type '<class 'numpy.float64'>'
	with 840 stored elements in Compressed Sparse Row format>

In [61]:
def get_matches_df(sparse_matrix, name_vector, top=840):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similarity': similairity})

matches_df = get_matches_df(matches, name_address)

In [62]:
matches_df.to_csv('xx.csv')

In [63]:
matches_df[matches_df['similarity'] < 0.99999].sort_values(by=['similarity'], ascending=False).head(30)

,left_side,right_side,similarity
826,Pike's Place Lux Suites by Barsala 2nd Ave and...,Pike's Place Lux Suites by Barsala 2rd Ave and...,0.734016
831,Pike's Place Lux Suites by Barsala 2rd Ave and...,Pike's Place Lux Suites by Barsala 2nd Ave and...,0.734016
256,Holiday Inn Express & Suites Seattle City Cent...,Holiday Inn Express & Suites Seattle-City Cent...,0.712321
206,Holiday Inn Express & Suites Seattle-City Cent...,Holiday Inn Express & Suites Seattle City Cent...,0.712321
181,Travelodge Seattle by The Space Needle 200 6th...,Travelodge Seattle by The Space Needle 200 6th...,0.669974
211,Travelodge Seattle by The Space Needle 200 6th...,Travelodge Seattle by The Space Needle 200 6th...,0.669974
836,citizenM Seattle South Lake Union hotel 201 We...,citizenM Seattle South Lake Union hotel 201 We...,0.651961
791,citizenM Seattle South Lake Union hotel 201 We...,citizenM Seattle South Lake Union hotel 201 We...,0.651961
586,Quality Inn & Suites Seattle Center 618 John S...,Quality Inn & Suites Seattle Center 618 John S...,0.627400
551,Quality Inn & Suites Seattle Center 618 John S...,Quality Inn & Suites Seattle Center 618 John S...,0.627400


In [57]:
matches_df[matches_df['similarity'] < 0.50].right_side.nunique()

152

In [30]:
xx = matches_df.to_csv('xx.csv')

In [18]:
xx